In [ ]:
pip install pydicom

In [2]:
import numpy as np
import pydicom
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model

In [ ]:
def load_dicom_data(root_folder):
    dicom_data = []

    for root, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".dcm"):
                file_path = os.path.join(root, file)
                dicom = pydicom.dcmread(file_path)
                dicom_array = dicom.pixel_array

                if len(dicom_data) == 0 or dicom_array.shape == dicom_data[0].shape:
                    dicom_data.append(dicom_array)
                else:
                    print(f"Ignoring file {file} due to mismatched dimensions.")

    return np.array(dicom_data)

data_folder = '/content/drive/MyDrive/Data'
dicom_data = load_dicom_data(data_folder)

print(f"Loaded {len(dicom_data)} DICOM files.")

In [4]:
labels = np.random.randint(2, size=len(dicom_data))
X = dicom_data[:, :, :, np.newaxis]
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
target_resolution = (224, 224)
X_train_rgb = np.repeat(X_train, 3, axis=-1)
X_test_rgb = np.repeat(X_test, 3, axis=-1)
X_train_resized = np.array([cv2.resize(img, target_resolution) for img in X_train_rgb])
X_test_resized = np.array([cv2.resize(img, target_resolution) for img in X_test_rgb])

In [7]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=target_resolution + (3,))

model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
batch_size = 8

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

In [ ]:
train_generator = train_datagen.flow(X_train_resized, y_train, batch_size=batch_size)

history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train_resized) // batch_size,
    epochs=20,
    validation_data=(X_test_resized, y_test))

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()